## 1 - Setup

Nesta seção configuramos caminhos padronizados para salvar o modelo (.keras) e os mapeamentos (.pkl) dentro desta versão.
Esses caminhos garantem reprodutibilidade e ajudam a manter o repositório organizado.

In [ ]:
# SETUP_ARTIFACT_PATHS
from pathlib import Path
BASE_DIR = Path.cwd().resolve()
if BASE_DIR.name == 'notebooks':
    BASE_DIR = BASE_DIR.parent
MODELS_DIR = BASE_DIR / 'models'
MAPPINGS_DIR = BASE_DIR / 'mappings'
MODELS_DIR.mkdir(parents=True, exist_ok=True)
MAPPINGS_DIR.mkdir(parents=True, exist_ok=True)
MODELO_OUT = MODELS_DIR / 'modelo_char_rnn.keras'
MAPEAMENTOS_OUT = MAPPINGS_DIR / 'mapeamentos.pkl'
print('Modelo:', MODELO_OUT)
print('Mapeamentos:', MAPEAMENTOS_OUT)


## 2 - Dados e Pre-Processamento

Carregamos o corpus (Dom Casmurro, Project Gutenberg) e aplicamos normalizações simples:
- converter para minúsculas; 
- comprimir espaços em branco.
Essas normalizações reduzem o vocabulário e o ruído, melhorando o aprendizado em modelos por caracteres.

In [ ]:
import tensorflow as tf, os, pickle
URL_LIVRO = 'https://www.gutenberg.org/files/55752/55752-0.txt'
caminho_arquivo = tf.keras.utils.get_file('dom_casmurro.txt', URL_LIVRO)
with open(caminho_arquivo, 'rb') as f:
    texto = f.read().decode('utf-8', errors='ignore')
# Normalizacao
texto = texto.lower()
texto = ' '.join(texto.split())
print('Tamanho do corpus (chars):', len(texto))


## 3 - Vocabulario e Janelas de Treino

Criamos o vocabulário de caracteres e os mapeamentos `char→id` e `id→char` (salvos em `MAPEAMENTOS_OUT`).

Também definimos o comprimento da janela de entrada (tamanho de sequência) e preparamos um gerador de lotes que cria as janelas de treino e os rótulos (próximo caractere) sob demanda, economizando memória.

In [ ]:
import numpy as np
TAMANHO_SEQUENCIA = 160
caracteres = sorted(list(set(texto)))
char_para_int = {c: i for i, c in enumerate(caracteres)}
int_para_char = {i: c for i, c in enumerate(caracteres)}
n_vocabulario = len(caracteres)
with open(MAPEAMENTOS_OUT, 'wb') as f:
    pickle.dump({'char_para_int': char_para_int, 'int_para_char': int_para_char, 'tamanho_sequencia': TAMANHO_SEQUENCIA}, f)
print('Vocabulario:', n_vocabulario, 'caracteres')

def batch_generator(texto, char2idx, seq_len=160, batch_size=256, step=1):
    n = len(texto) - seq_len
    vocab = len(char2idx)
    i = 0
    while True:
        X = np.zeros((batch_size, seq_len, vocab), dtype=np.float32)
        y = np.zeros((batch_size,), dtype=np.int32)
        for b in range(batch_size):
            idx = (i + b*step) % n
            seq = texto[idx: idx+seq_len]
            nxt = texto[idx+seq_len]
            for t, ch in enumerate(seq):
                X[b, t, char2idx.get(ch, 0)] = 1.0
            y[b] = char2idx.get(nxt, 0)
        i = (i + batch_size*step) % n
        yield X, y

steps_per_epoch = max(1, (len(texto) - TAMANHO_SEQUENCIA) // 256)
print('steps_per_epoch (aprox):', steps_per_epoch)


## 4 - Arquitetura do Modelo

Usamos uma LSTM seguida de `Dense(vocab, softmax)`. A entrada é one-hot (seq_len × vocab).

Esta arquitetura simples é adequada para fins didáticos de LM por caracteres.

In [ ]:
from tensorflow.keras import models, layers, optimizers
model = models.Sequential([
    layers.LSTM(256, input_shape=(TAMANHO_SEQUENCIA, n_vocabulario)),
    layers.Dense(n_vocabulario, activation='softmax'),
])
optimizer = optimizers.Adam(learning_rate=2e-3, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


## 5 - Treinamento

Treinamos com callbacks para manter o melhor modelo e ajustar a taxa de aprendizado:
- ModelCheckpoint (salva o melhor); 
- ReduceLROnPlateau (reduz LR em platôs);
- EarlyStopping (para cedo e restaura os melhores pesos).

In [ ]:
from tensorflow.keras import callbacks as kb
BATCH_SIZE = 256
EPOCAS_TREINO = 40
monitor='loss'
cb = [
    kb.ModelCheckpoint(str(MODELO_OUT), save_best_only=True, monitor=monitor, mode='min'),
    kb.ReduceLROnPlateau(monitor=monitor, factor=0.5, patience=3, min_lr=5e-5),
    kb.EarlyStopping(monitor=monitor, patience=5, restore_best_weights=True),
]
gen = batch_generator(texto, char_para_int, seq_len=TAMANHO_SEQUENCIA, batch_size=BATCH_SIZE, step=1)
steps_per_epoch = max(1, (len(texto) - TAMANHO_SEQUENCIA) // BATCH_SIZE)
history = model.fit(gen, steps_per_epoch=steps_per_epoch, epochs=EPOCAS_TREINO, callbacks=cb, verbose=1)
print('Treinamento concluido! Melhor modelo salvo em:', MODELO_OUT)


## 6 - Salvamento e Carregamento

Os artefatos ficam em `models/` e `mappings/` desta versão. Abaixo um exemplo de recarregamento para geração sem retreinar.

In [ ]:
import tensorflow as tf, pickle
modelo = tf.keras.models.load_model(MODELO_OUT) if MODELO_OUT.exists() else model
with open(MAPEAMENTOS_OUT, 'rb') as f:
    maps = pickle.load(f)
c2i = maps['char_para_int']
i2c = maps['int_para_char']
print('Artefatos prontos para gerar texto.')


## 7 - Geracao de Texto

Usamos amostragem top-k (k=20) por padrão, com temperatura 0.8. Forneça uma seed (~160 caracteres) do próprio corpus para melhor fluência.

Conceitos:
- Temperatura controla a aleatoriedade (0.7–0.9 sugerido);
- Top-k restringe a escolha aos k tokens mais prováveis, evitando repetições.

In [ ]:
import numpy as np
def sample_top_k(probs, k=20, temperature=0.8, rng=None):
    probs = np.asarray(probs, dtype=np.float64)
    if rng is None: rng = np.random.default_rng()
    k = int(max(1, min(k, probs.size)))
    top_idx = np.argpartition(-probs, k-1)[:k]
    sel = probs[top_idx]
    if temperature > 0:
        logits = np.log(np.maximum(sel, 1e-9)) / temperature
        logits -= logits.max()
        sel = np.exp(logits)
    p = sel / sel.sum()
    return int(top_idx[rng.choice(len(top_idx), p=p)])

def gerar_texto(model, char_to_id, id_to_char, seed, comprimento=400, k=20, temperatura=0.8):
    vocab = len(id_to_char)
    seq_len = model.input_shape[1] if isinstance(model.input_shape, (list,tuple)) else 160
    rep = ' ' if ' ' in char_to_id else next(iter(char_to_id.keys()))
    seed = ''.join(ch if ch in char_to_id else rep for ch in seed)
    ids = [char_to_id[ch] for ch in seed][-seq_len:]
    if len(ids) < seq_len:
        pad = char_to_id.get(' ', ids[0] if ids else 0)
        ids = [pad] * (seq_len - len(ids)) + ids
    out = []
    rng = np.random.default_rng(42)
    for _ in range(comprimento):
        X = np.zeros((1, seq_len, vocab), dtype=np.float32)
        for t, idx in enumerate(ids):
            if idx < vocab: X[0, t, idx] = 1.0
        p = model.predict(X, verbose=0)[0]
        nxt = sample_top_k(p, k=k, temperature=temperatura, rng=rng)
        out.append(id_to_char.get(nxt, '?'))
        ids = ids[1:] + [nxt]
    return ''.join(out)

# Exemplo de uso (edite 'seed' com ~160 chars do corpus):
# seed = '...'
# print(gerar_texto(modelo, c2i, i2c, seed, comprimento=400, k=20, temperatura=0.8))
